In [ ]:
!pip install python_speech_features

In [4]:
import numpy as np
import pandas as pd
import os
from acoustics import acstc_anlys
from utilities import get_voiced
import time

def run_analysis_single(f, pr_script, tg_script, get_acoustics=True):
    """
    Analyze audio file and extract acoustic features.

    Parameters
    ----------
    f : str
        Filepath to -converted.wav audio file (absolute path). MUST be the audio file.
    pr_script : str
        Praat script for extracting voiced segments from audio data (absolute path).
    tg_script : str
        Praat script for textgrid parsing for timing analysis (absolute path).
    get_acoustics : bool
        Extract acoustic features?

    Returns
    -------
    output : pd.DataFrame
        Single-row dataframe containing all extracted features.
    """
    # Verify inputs
    assert (type(pr_script) == str) and (len(pr_script) > 3), "Please specify location of voice extraction Praat script"
    assert (type(tg_script) == str) and (len(tg_script) > 3), "Please specify location of TextGrid script"

    # Normalize file paths
    file = f.replace("\\", "/")
    data_dir = os.path.dirname(file).replace("\\", "/")

    # Path for voiced file
    fnew_test = os.path.join(data_dir, "voiced", os.path.basename(file).replace(".wav", "_OnlyVoiced.wav"))
    
    if get_acoustics:
        # Check if the voiced file already exists
        if os.path.isfile(fnew_test):
            print(f"Voiced file already extracted: {fnew_test}")
        else:
            print(f"Extracting voiced segments for {file}...")
            get_voiced(file=file,
                       data_dir=data_dir,
                       praat_script=pr_script,
                       textgrid_script=tg_script)
        
        # Extract acoustic features
        print(f"Extracting acoustic features for {file}...")
        output = acstc_anlys(file=file,
                             data_dir=data_dir,
                             f0_min=75,
                             f0_max=600)

        # Set the row index to the file name (without extension)
        output.index = [os.path.basename(file).split('.')[0]]

    return output    


# Load configuration
config_file_path = '/Users/edward/Documents/GitHub/als-project/edward_config.config'
if not os.path.exists(config_file_path):
    raise FileNotFoundError(f"Config file '{config_file_path}' does not exist. Please ensure the path is correct.")

try:
    cfg = pd.read_csv(config_file_path, header=None, index_col=0)
    run_home = cfg.loc["run_home"].values[0].strip().strip('"')  # Remove extra spaces or quotes
    tg_script = cfg.loc["textgrid_script"].values[0].strip().strip('"')
    pr_script = cfg.loc["praat_script"].values[0].strip().strip('"')

    # Validate paths
    if not os.path.exists(run_home):
        raise FileNotFoundError(f"Run home directory '{run_home}' does not exist. Please check the config file.")
    if not os.path.exists(tg_script):
        raise FileNotFoundError(f"TextGrid script '{tg_script}' does not exist.")
    if not os.path.exists(pr_script):
        raise FileNotFoundError(f"Praat script '{pr_script}' does not exist.")

    os.chdir(run_home)
except Exception as e:
    raise RuntimeError(f"Error loading configuration: {e}")

# Load metadata
metadata_path = "/Users/edward/Documents/GitHub/als-project/filtered/data/filtered_final_consolidated_dataset_Fletcher.csv"
if not os.path.exists(metadata_path):
    raise FileNotFoundError(f"Metadata file '{metadata_path}' does not exist.")

df = pd.read_csv(metadata_path)

# Verify `voiced_file_path` column exists
if "filtered_file_path" not in df.columns:
    raise KeyError("The column 'voiced_file_path' does not exist in the metadata. Please verify the CSV structure.")

# Correct paths to prepend `filtered/` properly
df["resolved_path"] = df["filtered_file_path"].apply(
    lambda x: os.path.join(run_home, "filtered", x) if not x.startswith("filtered/") else os.path.join(run_home, x)
)

# Get column names by analyzing one sample file
try:
    first_path = df.loc[0, "resolved_path"]
    if not os.path.exists(first_path):
        raise FileNotFoundError(f"File {first_path} does not exist. Please check your metadata paths.")
    sample_output = run_analysis_single(first_path, pr_script, tg_script)
    new_columns = df.columns.to_list() + sample_output.columns.to_list()
except Exception as e:
    raise RuntimeError(f"Error during analysis of sample file: {e}")

# Initialize lists for processed data and errors
combined_rows = []
errors = []

# Process files from the metadata
count = 1
df_subset = df.iloc[948:]  # Adjust range as needed

for index, row in df_subset.iterrows():
    try:
        file_path = row['resolved_path']
        if not os.path.exists(file_path):
            print(f"File not found: {file_path}")
            errors.append({"file": file_path, "error": "File not found"})
            continue

        print(f"Processing {count}: {os.path.basename(file_path)}")

        out = run_analysis_single(file_path, pr_script, tg_script)
        combined_row = list(row) + list(out.iloc[0, :])
        combined_rows.append(combined_row)

        # Save intermediate results periodically
        if count % 10 == 0:
            total_output = pd.DataFrame(combined_rows, columns=new_columns)
            total_output.to_csv("metadata_acoustic_clinical.csv", index=False)
        
        count += 1
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        errors.append({"file": file_path, "error": str(e)})

# Save final results
total_output = pd.DataFrame(combined_rows, columns=new_columns)
total_output.to_csv("metadata_acoustic_clinical.csv", index=False)

# Save errors
error_df = pd.DataFrame(errors)
error_df.to_csv("error_files.csv", index=False)

print(f"Processing complete. Processed {count - 1} files. Errors: {len(errors)}")


Processing 32: MSA8a1_clean_OnlyVoiced.wav
Voiced file already extracted: /Users/edward/Documents/GitHub/als-project/filtered/data/MSA_dataset/MSA_a_1/voiced/voiced/MSA8a1_clean_OnlyVoiced_OnlyVoiced.wav
Extracting acoustic features for /Users/edward/Documents/GitHub/als-project/filtered/data/MSA_dataset/MSA_a_1/voiced/MSA8a1_clean_OnlyVoiced.wav...
INTENSITY: Detected delta >1ms
INTENSITY: Detected delta >1ms
Interpolated 2 value(s)
Interpolated 2 value(s)
Interpolated 2 value(s)
Processing 33: MSA3a1_clean_OnlyVoiced.wav
Voiced file already extracted: /Users/edward/Documents/GitHub/als-project/filtered/data/MSA_dataset/MSA_a_1/voiced/voiced/MSA3a1_clean_OnlyVoiced_OnlyVoiced.wav
Extracting acoustic features for /Users/edward/Documents/GitHub/als-project/filtered/data/MSA_dataset/MSA_a_1/voiced/MSA3a1_clean_OnlyVoiced.wav...
INTENSITY: Detected delta >1ms
INTENSITY: Detected delta >1ms
Interpolated 2 value(s)
Interpolated 2 value(s)
Interpolated 2 value(s)
Processing 34: MSA4a1_clean_

/opt/anaconda3/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


INTENSITY: Detected delta >1ms
INTENSITY: Detected delta >1ms
Interpolated 1 value(s)
Interpolated 1 value(s)
Interpolated 1 value(s)
Processing 45: MSA16i2_clean_OnlyVoiced.wav
Voiced file already extracted: /Users/edward/Documents/GitHub/als-project/filtered/data/MSA_dataset/MSA_i_2/voiced/voiced/MSA16i2_clean_OnlyVoiced_OnlyVoiced.wav
Extracting acoustic features for /Users/edward/Documents/GitHub/als-project/filtered/data/MSA_dataset/MSA_i_2/voiced/MSA16i2_clean_OnlyVoiced.wav...
INTENSITY: Detected delta >1ms
INTENSITY: Detected delta >1ms
Interpolated 2 value(s)
Interpolated 2 value(s)
Interpolated 2 value(s)
Processing 46: MSA16i1_clean_OnlyVoiced.wav
Voiced file already extracted: /Users/edward/Documents/GitHub/als-project/filtered/data/MSA_dataset/MSA_i_2/voiced/voiced/MSA16i1_clean_OnlyVoiced_OnlyVoiced.wav
Extracting acoustic features for /Users/edward/Documents/GitHub/als-project/filtered/data/MSA_dataset/MSA_i_2/voiced/MSA16i1_clean_OnlyVoiced.wav...
INTENSITY: Detected de

/opt/anaconda3/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


INTENSITY: Detected delta >1ms
Interpolated 2 value(s)
Interpolated 2 value(s)
Interpolated 2 value(s)
Error processing file /Users/edward/Documents/GitHub/als-project/filtered/data/MSA_dataset/MSA_i_2/voiced/MSA11i1_clean_OnlyVoiced.wav: Input X contains NaN.
PCA does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values
Processing 55: MSA18i2_clean_OnlyVoiced.wav
Voiced file already extracted: /Users/edward/Documents/GitHub/als-project/filtered/data/MSA

...

INTENSITY: Detected delta >1ms
INTENSITY: Detected delta >1ms
Processing 791: VA1GMIAOSVI44M100220170946_OnlyVoiced.wav
Voiced file already extracted: /Users/edward/Documents/GitHub/als-project/filtered/data/Italian_Parkinsons_Voice_and_speech/28 People with Parkinson's disease/17-28/Giovanni M/voiced/voiced/VA1GMIAOSVI44M100220170946_OnlyVoiced_OnlyVoiced.wav
Extracting acoustic features for /Users/edward/Documents/GitHub/als-project/filtered/data/Italian_Parkinsons_Voice_and_speech/28 People with Parkinson's disease/17-28/Giovanni M/voiced/VA1GMIAOSVI44M100220170946_OnlyVoiced.wav...
INTENSITY: Detected delta >1ms
INTENSITY: Detected delta >1ms
Interpolated 2 value(s)
Interpolated 2 value(s)
Interpolated 2 value(s)
Processing 792: VE2GMIAOSVI44M100220170947_OnlyVoiced.wav
Voiced file already extracted: /Users/edward/Documents/GitHub/als-project/filtered/data/Italian_Parkinsons_Voice_and_speech/28 People with Parkinson's disease/17-28/Giovanni M/voiced/voiced/VE2GMIAOSVI44M10022017094

/Users/edward/Documents/GitHub/als-project/acoustics.py:98: RuntimeWarning: Mean of empty slice
  formant_dict[f"f{i}_mean"] = np.nanmean(formant_value)
/Users/edward/Documents/GitHub/als-project/acoustics.py:99: RuntimeWarning: All-NaN slice encountered
  formant_dict[f"f{i}_median"] = np.nanmedian(formant_value)  #######
/opt/anaconda3/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/anaconda3/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1545: RuntimeWarning: All-NaN slice encountered
  return _nanquantile_unchecked(


INTENSITY: Detected delta >1ms
INTENSITY: Detected delta >1ms
Interpolated 2 value(s)
Interpolated 2 value(s)
Interpolated 2 value(s)
Processing 803: VI1AGNUTGOL52F100220171051_OnlyVoiced.wav
Voiced file already extracted: /Users/edward/Documents/GitHub/als-project/filtered/data/Italian_Parkinsons_Voice_and_speech/28 People with Parkinson's disease/17-28/Antonia G/voiced/voiced/VI1AGNUTGOL52F100220171051_OnlyVoiced_OnlyVoiced.wav
Extracting acoustic features for /Users/edward/Documents/GitHub/als-project/filtered/data/Italian_Parkinsons_Voice_and_speech/28 People with Parkinson's disease/17-28/Antonia G/voiced/VI1AGNUTGOL52F100220171051_OnlyVoiced.wav...
INTENSITY: Detected delta >1ms
INTENSITY: Detected delta >1ms
Interpolated 1 value(s)
Interpolated 1 value(s)
Interpolated 1 value(s)
Processing 804: VO2AGNUTGOL52F100220171053_OnlyVoiced.wav
Voiced file already extracted: /Users/edward/Documents/GitHub/als-project/filtered/data/Italian_Parkinsons_Voice_and_speech/28 People with Parkin

/Users/edward/Documents/GitHub/als-project/acoustics.py:305: RuntimeWarning: All-NaN slice encountered
  formant_slopes_dict[f"f{formant_num}_d_dx_median"] = np.nanmedian(formant_slope)
/opt/anaconda3/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1545: RuntimeWarning: All-NaN slice encountered
  return _nanquantile_unchecked(
/Users/edward/Documents/GitHub/als-project/acoustics.py:255: RuntimeWarning: All-NaN slice encountered
  formant_accels_dict[f"f{formant_num}_dd_dx_median"] = np.nanmedian(


INTENSITY: Detected delta >1ms
INTENSITY: Detected delta >1ms
Interpolated 1 value(s)
Interpolated 1 value(s)
Interpolated 1 value(s)
Processing 825: VA1NCIICAOC52M100220171239_OnlyVoiced.wav
Voiced file already extracted: /Users/edward/Documents/GitHub/als-project/filtered/data/Italian_Parkinsons_Voice_and_speech/28 People with Parkinson's disease/17-28/Nicolò C/voiced/voiced/VA1NCIICAOC52M100220171239_OnlyVoiced_OnlyVoiced.wav
Extracting acoustic features for /Users/edward/Documents/GitHub/als-project/filtered/data/Italian_Parkinsons_Voice_and_speech/28 People with Parkinson's disease/17-28/Nicolò C/voiced/VA1NCIICAOC52M100220171239_OnlyVoiced.wav...
INTENSITY: Detected delta >1ms
INTENSITY: Detected delta >1ms
Interpolated 2 value(s)
Interpolated 2 value(s)
Interpolated 2 value(s)
Processing 826: VA2NCIICAOC52M100220171240_OnlyVoiced.wav
Voiced file already extracted: /Users/edward/Documents/GitHub/als-project/filtered/data/Italian_Parkinsons_Voice_and_speech/28 People with Parkinso

In [ ]:
import os
import pandas as pd

# Path to the configuration file
config_file_path = '/Users/edward/Documents/GitHub/als-project/edward_config.config'

# Debugging: Check if the config file exists
if not os.path.exists(config_file_path):
    raise FileNotFoundError(f"Config file '{config_file_path}' does not exist. Please verify the path.")

try:
    # Load the configuration file
    cfg = pd.read_csv(config_file_path, header=None, index_col=0)
    run_home = cfg.loc["run_home"].values[0].strip().strip('"')  # Remove surrounding quotes
    tg_script = cfg.loc["textgrid_script"].values[0].strip().strip('"')  # Remove surrounding quotes
    pr_script = cfg.loc["praat_script"].values[0].strip().strip('"')  # Remove surrounding quotes

    # Debugging: Print loaded paths
    print(f"Run home directory: {run_home}")
    print(f"TextGrid script path: {tg_script}")
    print(f"Praat script path: {pr_script}")

    # Validate if paths exist
    if not os.path.exists(run_home):
        raise FileNotFoundError(f"Run home directory '{run_home}' does not exist. Please check the config file.")
    if not os.path.exists(tg_script):
        raise FileNotFoundError(f"TextGrid script '{tg_script}' does not exist. Please check the config file.")
    if not os.path.exists(pr_script):
        raise FileNotFoundError(f"Praat script '{pr_script}' does not exist. Please check the config file.")

    print("Config file loaded successfully and all paths validated.")

except FileNotFoundError as e:
    print(e)
    raise
except KeyError as e:
    print(f"Key error: {e}. Check if the required keys are present in the config file.")
    raise
except Exception as e:
    print(f"An unexpected error occurred: {e}")
    raise
